##### humidity scrap

In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
import pandas as pd 
import re 

opts = Options()
opts.set_headless()
assert opts.headless  # Operating in headless mode

<ipython-input-1-79c39fba795c>:7: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [2]:
driver = Firefox(options=opts)
#driver.implicitly_wait(1)
#driver.get('https://www.tutiempo.net/clima/02-1983/ws-81810.html')

In [3]:
fechas=pd.read_csv('fechas.csv',index_col=[0])
dates=list(fechas['fecha'])
dates=set(dates)
dates_bis=list(dates)
len(dates_bis)

442

In [280]:
for dt in dates_bis[2:5]:
    url='https://www.tutiempo.net/clima/'+dt+'/ws-81810.html'
    print(url)
    print(dt)

https://www.tutiempo.net/clima/01-2006/ws-81810.html
01-2006
https://www.tutiempo.net/clima/03-1987/ws-81810.html
03-1987
https://www.tutiempo.net/clima/08-1995/ws-81810.html
08-1995


In [4]:
counts=1

In [5]:
data,humitat =[],[]

In [ ]:
# some of the data is coded so it needs to be srcaped in different ways
for dt in dates_bis:
    url='https://www.tutiempo.net/clima/'+dt+'/ws-81810.html'
    driver.get(url)
    print(counts)

    mes= driver.find_element_by_xpath("//div[@class='titulo']//span").get_attribute("textContent")
    n=2
    while n<33:
        try:
            if n==5 or n==6 or n==7 or n==11 or n==12 or n==13 or n==17 or n==18 or n==19 or n==23 or n==24 or n==25 or n==29 or n==30 or n==31:
                a1= str('//table[@class=\'medias mensuales numspan\']//tr['+str(n)+']/td[6]/span[1]')
                a2= str('//table[@class=\'medias mensuales numspan\']//tr['+str(n)+']/td[6]/span[2]')
                prim=dd=driver.find_element_by_xpath(a1).get_attribute("class")
                seco=dd=driver.find_element_by_xpath(a2).get_attribute("class")
                data.append(mes + '-'+  str(n-1))
                humitat.append(prim + ' ' + seco)
                n=n+1
            else:
                address= str('//table[@class=\'medias mensuales numspan\']//tr['+str(n)+']//td[6]')
                hum=driver.find_element_by_xpath(address).get_attribute("textContent")
                data.append(mes + '-'+  str(n-1))
                humitat.append(hum)
                n=n+1
        except:
            print('')
            n=n+1
    counts=counts+1


In [298]:

len(humitat)

13412

In [ ]:
ntde=0 nthi=1 ntpq=2 ntxy=3 ntgg=4 ntaz=5 ntyy=6 ntxo=7 ntre=8 ntll=9

In [47]:
humitatdf = pd.DataFrame(list(zip(data,humitat)), columns =['date','humidity'])

humitatdf

,date,humidity
0,Marzo - 2006-1,65
1,Marzo - 2006-2,62
2,Marzo - 2006-3,53
3,Marzo - 2006-4,ntgy ntaa
4,Marzo - 2006-5,ntgy ntza
...,...,...
13407,Julio - 1987-27,60
13408,Julio - 1987-28,ntpo ntpo
13409,Julio - 1987-29,ntpo ntjg
13410,Julio - 1987-30,nthy ntef


In [48]:
humitatdf.to_csv('humitat.csv')

In [14]:
humitatdf=pd.read_csv('humitat.csv',index_col=[0])
humitatdf

,date,humidity
0,Marzo - 2006-1,65
1,Marzo - 2006-2,62
2,Marzo - 2006-3,53
3,Marzo - 2006-4,ntgy ntaa
4,Marzo - 2006-5,ntgy ntza
...,...,...
13407,Julio - 1987-27,60
13408,Julio - 1987-28,ntpo ntpo
13409,Julio - 1987-29,ntpo ntjg
13410,Julio - 1987-30,nthy ntef


In [15]:
humitatdf[['month', 'year','day']] = humitatdf['date'].str.split('-', 2, expand=True)

In [16]:
humitatdf.drop(['date'], axis = 1, inplace=True)
humitatdf

,humidity,month,year,day
0,65,Marzo,2006,1
1,62,Marzo,2006,2
2,53,Marzo,2006,3
3,ntgy ntaa,Marzo,2006,4
4,ntgy ntza,Marzo,2006,5
...,...,...,...,...
13407,60,Julio,1987,27
13408,ntpo ntpo,Julio,1987,28
13409,ntpo ntjg,Julio,1987,29
13410,nthy ntef,Julio,1987,30


In [17]:
humitatdf['month'].unique()

array(['Marzo ', 'Noviembre ', 'Mayo ', 'Julio ', 'Agosto ', 'Junio ',
       'Enero ', 'Febrero ', 'Abril ', 'Octubre ', 'Diciembre ',
       'Septiembre '], dtype=object)

In [18]:
# turn months into numbers
humitatdf=humitatdf.replace({'month' : { 'Marzo' : '03','Julio' : '07','Noviembre' : '11','Mayo' : '05',
                                        'Agosto' : '08','Enero' : '01','Febrero' : '02','Abril' : '04',
                                        'Junio' : '06','Octubre' : '10','Diciembre' : '12',
                                        'Septiembre' : '09' }}, regex=True)

In [25]:
humitatdf[0:10]

,humidity,month,year,day
0,65,03,2006,1
1,62,03,2006,2
2,53,03,2006,3
3,64,03,2006,4
4,63,03,2006,5
5,43,03,2006,6
6,67,03,2006,7
7,74,03,2006,8
8,70,03,2006,9
9,46,03,2006,10


In [20]:
humitatdf=humitatdf.replace({'humidity' : { ' ':'-' }}, regex=True)

##### decoding the special characters from the site

In [21]:
#'ntox':'4'
humitatdf=humitatdf.replace({'humidity' : { 'ntgy':'6','ntaa':'4' ,'ntza':'3','ntox':'7',
                                          'ntbz':'5','ntbc':'0','ntqr':'8','ntrs':'2',
                                          'ntnt':'9','ntjk':'1','ntyy':'6','ntab':'0',
                                          'ntas':'8','ntcd':'3','ntfs':'7','nttn':'9',
                                          'ntzb':'5','nthj':'6','ntpo':'7','ntef':'3', 'ntdr':'6','ntjg':'9',
                                          'ntgo':'5','nthy':'8','ntpo':'7','ntfg':'0' , 'ntno':'1',
                                          'ntbb':'4'}}, regex=True)

In [22]:

humitatdf=humitatdf.replace({'humidity' : {'ntpq': '2', 'ntll':'9', 'ntre':'8' , 'ntgg':'4', 'ntxo':'7','ntre':'8',
                                           'ntaz':'5' , 'nthi':'1','ntxy':'3','ntlm':'1', 'nttu':'2','ntde':'0',
                                          'ntvw':'2','ntee':'4'}}, regex=True)
humitatdf=humitatdf.replace({'humidity' : {'-': ''}}, regex=True)                              

In [57]:
humitatdf1=humitatdf[['humidity','year','month','day']]

humitatdf1['day'] = humitatdf1['day'].apply(lambda x: x.zfill(2))

humitatdf1

,humidity,year,month,day
0,65,2006,03,01
1,62,2006,03,02
2,53,2006,03,03
3,64,2006,03,04
4,63,2006,03,05
...,...,...,...,...
13407,60,1987,07,27
13408,77,1987,07,28
13409,79,1987,07,29
13410,83,1987,07,30


In [59]:
cols = ['year','month','day']
humitatdf1['fecha'] = humitatdf1[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
humitatdf1=humitatdf1.replace({'fecha' : {' ': ''}}, regex=True)  
humitatdf1=humitatdf1[['humidity','fecha']]

In [60]:
humitatdf1

,humidity,fecha
0,65,2006-03-01
1,62,2006-03-02
2,53,2006-03-03
3,64,2006-03-04
4,63,2006-03-05
...,...,...
13407,60,1987-07-27
13408,77,1987-07-28
13409,79,1987-07-29
13410,83,1987-07-30


In [61]:
humitatdf1.to_csv('humidity.csv')